In [30]:
from seg_utils import *
#from preprocessing import crop_image
import segmentation_models_pytorch as smp
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
#import albumentations as albu

In [2]:
rgb = True
X_img, Y_mask = load_data(rgb = rgb, dim = (192, 240))

In [3]:
#print(X_img.shape)
#print(Y_mask.shape)

In [4]:
#Image.fromarray(Y_mask[37]*255).show()

In [80]:
augment_flip = True
augment_noise = 2

X_train, Y_train, X_val, Y_val, X_test, Y_test = divide_train_test(X_img, Y_mask, trainsize=0.8, 
                                                                   validationsize=0.1, rgb = rgb, 
                                                                   augment_flip = augment_flip, 
                                                                   augment_noise = augment_noise)

print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

Dividing data into train and test set.
Adding flipped images to trainset.
Adding images with noise to trainset.
Making Gauss1
Adding Gauss1
Making Gauss2
Adding Gauss2
(4200, 3, 192, 192)
(4200, 3, 192, 192)
(42, 3, 192, 192)
(42, 3, 192, 192)
(43, 3, 192, 192)
(43, 3, 192, 192)


In [81]:
X_train = torch.Tensor(X_train)
Y_train = torch.Tensor(Y_train)
X_val = torch.Tensor(X_val)
Y_val = torch.Tensor(Y_val)
X_test = torch.Tensor(X_test)
Y_test = torch.Tensor(Y_test)

In [82]:
print(X_train.shape)

torch.Size([4200, 3, 192, 192])


In [83]:
train = torch.utils.data.TensorDataset(X_train, Y_train)
val = torch.utils.data.TensorDataset(X_val, Y_val)
test = torch.utils.data.TensorDataset(X_test, Y_test)

In [84]:
train_loader = DataLoader(train, batch_size=8, shuffle=True, num_workers=12)
valid_loader = DataLoader(val, batch_size=1, shuffle=False, num_workers=4)

In [85]:

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

# Model

In [112]:
model = smp.Unet('resnet18', classes = 1, activation = 'sigmoid', encoder_weights='imagenet')

In [113]:
loss = smp.utils.losses.DiceLoss()

metrics = [
    smp.utils.metrics.IoU(threshold=0.5), #0.5
]

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)#, momentum=0.9)
#optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.1),])

In [114]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    #device='cuda',
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    #device='cuda',
    verbose=True,
)

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(0, 10):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
           
    if i == 5:
        optimizer.param_groups[0]['lr'] = 0.01
        print('Decrease decoder learning rate to 0.01!')  

        
        
        


Epoch: 0
valid: 100%|██████████| 42/42 [00:06<00:00,  6.88it/s, dice_loss - 0.9518, iou_score - 0.02736]
Model saved!

Epoch: 1
valid: 100%|██████████| 42/42 [00:06<00:00,  6.82it/s, dice_loss - 0.9105, iou_score - 0.04993]
Model saved!

Epoch: 2
valid: 100%|██████████| 42/42 [00:05<00:00,  7.48it/s, dice_loss - 0.9124, iou_score - 0.04729]

Epoch: 3
valid: 100%|██████████| 42/42 [00:05<00:00,  7.59it/s, dice_loss - 0.842, iou_score - 0.09812]
Model saved!

Epoch: 4
valid: 100%|██████████| 42/42 [00:05<00:00,  7.04it/s, dice_loss - 0.8269, iou_score - 0.1115] 
Model saved!

Epoch: 5
valid: 100%|██████████| 42/42 [00:05<00:00,  7.39it/s, dice_loss - 0.8791, iou_score - 0.07857]
Decrease decoder learning rate to 0.01!

Epoch: 6
train:  49%|████▉     | 257/525 [18:01<17:20,  3.88s/it, dice_loss - 0.8554, iou_score - 0.08094]

# Test best saved model

In [ ]:
# load best saved checkpoint
best_model = torch.load('./best_model.pth')

In [ ]:
test_dataloader = DataLoader(test)

# evaluate model on test set
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics,
    #device=DEVICE,
)

logs = test_epoch.run(test_dataloader)

In [ ]:

for i in range(5):
    n = np.random.choice(len(test))
    
    image_vis = test[n][0].numpy()
    image, gt_mask = test[n]
    image = image.numpy()
    gt_mask = gt_mask.numpy()
    
    gt_mask = gt_mask.squeeze()
    
    x_tensor = torch.from_numpy(image).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    
    #plt.subplot(1, 5, i + 1)
    plt.imshow(np.transpose(image_vis)+np.transpose(gt_mask)+np.transpose(pr_mask[np.newaxis]))
    #plt.imshow(np.transpose(gt_mask), cmap='jet', alpha=0.5)
    plt.show()

    #plt.imshow(np.transpose(image_vis)+np.transpose(pr_mask[np.newaxis]))
    #plt.imshow(np.transpose(pr_mask), cmap='jet', alpha=0.5)
    #plt.show()
    
    '''
    visualize(
        image=np.transpose(image_vis), 
        ground_truth_mask=np.transpose(gt_mask), 
        predicted_mask=np.transpose(pr_mask)
    )'''
